In [54]:
import numpy as np
import pandas as pd

mi target será la variable sales, significa cantidad de ventas en ese dia

Exogenas:

* Dia de la semana
* feriados
* tipo (store)
* cluster (store)

## Tablas

### train

In [55]:
df_train= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/train.csv')
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


### Test (usar para testear)

In [56]:
df_test= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/test.csv')
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


### Transacciones (no usar)

In [57]:
df_transaccion= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/transactions.csv')
df_transaccion

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


### Tiendas

In [58]:
df_store= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/stores.csv')
df_store.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [59]:
# Cantidad de tiendas
df_store['store_nbr'].count()

54

In [60]:
df_store.loc[df_store['type']=='B']

,store_nbr,city,state,type,cluster
8,9,Quito,Pichincha,B,6
10,11,Cayambe,Pichincha,B,6
17,18,Quito,Pichincha,B,16
19,20,Quito,Pichincha,B,6
20,21,Santo Domingo,Santo Domingo de los Tsachilas,B,6
30,31,Babahoyo,Los Rios,B,10
33,34,Guayaquil,Guayas,B,6
38,39,Cuenca,Azuay,B,6


In [61]:
df_store['cluster'].unique()

array([13,  8,  9,  4,  6, 15,  7,  3, 12, 16,  1, 10,  2,  5, 11, 14, 17],
      dtype=int64)

In [62]:
df_store['type'].unique()

array(['D', 'B', 'C', 'E', 'A'], dtype=object)

### Dias Feriados

In [63]:
df_holidays_events= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/holidays_events.csv')
df_holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


### Precio petroleo

In [64]:
df_oil= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/4to cuatrimestre/PP2/Series de Tiempo/data/oil.csv')
df_oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


## EDA

### Train

In [65]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [66]:
#Podemos observar que no tenemos datos nulos en todo nuestro DataSet
df_train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [67]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [68]:
#Para facilitar el trabajo con las fechas, pasamos la columna date a tipo datetime
df_train['date']=pd.to_datetime(df_train['date'])

#### Cant de ventas por dia

##### Por dia

In [69]:
# Agrupo por columna date y famili, sumando sales de las diferentes stores
df_cant_ventas=df_train.groupby(['date','family'])['sales'].sum().reset_index()
# Pongo a date como indice para usar una herramienta en un futuro
df_cant_ventas.index= df_cant_ventas['date']
df_cant_ventas.head()

,date,family,sales
date,,,
2013-01-01,2013-01-01,AUTOMOTIVE,0.0
2013-01-01,2013-01-01,BABY CARE,0.0
2013-01-01,2013-01-01,BEAUTY,2.0
2013-01-01,2013-01-01,BEVERAGES,810.0
2013-01-01,2013-01-01,BOOKS,0.0


In [70]:
import plotly.express as px
fig = px.line(df_cant_ventas, x=df_cant_ventas.index, y='sales',color='family')
fig.show()

Se puede observar que todos los 1 de enero, indeferente al año, no se realizan casi ventas

##### Por semanas

In [71]:
# Agrupo el DataFrame df_cant_ventas por semana, promediando sus valores
df_cant_ventas_semana=df_cant_ventas.resample('SM').mean()
df_cant_ventas_semana.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13632\3387147684.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,sales
date,
2012-12-31,10559.603173
2013-01-15,9806.503510
2013-01-31,10512.203117
2013-02-15,10369.597110
2013-02-28,11043.817274


In [72]:
import plotly.express as px
fig = px.line(df_cant_ventas_semana, x=df_cant_ventas_semana.index, y='sales')
fig.show()

Otro análisis sin considerar la familia y agrupados por semanas, se puede observar que con el paso del tiempo, se fueron realizando mas ventas

### Oil

In [73]:
df_oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [74]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [75]:
df_oil['date']=pd.to_datetime(df_oil['date'])

In [76]:
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [77]:
import plotly.express as px
fig = px.line(df_oil, x='date', y='dcoilwtico')
fig.show()

Se observa que con el tiempo, el precio del peotroleo fue desendiendo, tambien se aprecia que hay varios dias donde no se cuenta con los datos del precio de petroleo, se imputaran en un futuro.

### Relacion cantidad de ventas y precio del petroleo

In [78]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [79]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [80]:
df_relacion= df_train.merge(df_oil,how='left')

In [81]:
df_relacion.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
sales               0
onpromotion         0
dcoilwtico     928422
dtype: int64

In [82]:
df_relacion=df_relacion.fillna(method='bfill')

In [83]:
df_relacion

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,93.14
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57


In [85]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_relacion['date'], y=df_relacion['sales'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=df_relacion['date'], y=df_relacion['dcoilwtico'],
                    mode='lines',
                    name='lines'))

fig.show()

KeyboardInterrupt: 